In [11]:
import pandas as pd
import numpy as np
from tika import parser
import os
import glob
import json
import jsonlines
import random
import re
from pprint import pprint

In [2]:
# making 500 text files
file = open('falsified_text_fromBIK.jsonl')
filename = 0

for i in file:
    res = json.loads(i)
    res = res["gens_article"]
    res = str(res)
    res = res.replace("[","")
    res = res.replace("]","")
    
    with open("./fake_text/"+str(filename)+'.txt', 'w',encoding="utf-8") as f:
        f.write(res)
        
    filename = filename + 1
    

In [22]:
#make it ready for the input for the discrimination file
with open('falsified_text_fromBIK.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    #pprint(result)

In [23]:
txt_files = glob.glob("*.txt")
txt_files = sorted(txt_files, key=lambda x: int(re.match('\D*(\d+)', x).group(1)))

print(len(txt_files))

214


In [24]:
txt_files[0]

'0.txt'

In [25]:
bik_df = pd.read_csv('../text_extraction/Bik_v2.tsv', sep='\t', encoding='unicode-escape')
bik_df.columns

Index(['Unnamed: 0', 'Authors', 'Title', 'Citation', 'DOI', 'Year', 'Month',
       '0', '1', '2', '3', 'FINDINGS', 'Reported', 'Correction Date',
       'Retraction', 'Correction', 'No Action', 'SUM \nCompleted', 'URL',
       'Home Site', 'Lab Size', 'Pub Rate', 'Other Journals',
       'First Author Affiliation', 'First Author Career Duration',
       'First Author Degree', 'First Author Degree Area', 'university_name',
       'world_rank_x', 'country_x', 'national_rank', 'quality_of_education',
       'alumni_employment', 'quality_of_faculty', 'publications', 'influence',
       'citations_x', 'broad_impact', 'patents', 'score', 'year_x',
       'world_rank_y', 'country_y', 'teaching', 'international', 'research',
       'citations_y', 'income', 'total_score', 'num_students',
       'student_staff_ratio', 'international_students', 'female_male_ratio',
       'year_y', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density',

In [30]:
master_file = []

for i in range(len(txt_files)):

    with open(txt_files[i]) as f:
        lines = f.readlines()
    
    newinput ={}
    
    newinput["title"] = str(bik_df['Title'][i])
    newinput['warc_date'] = '20220328'
    newinput["article"]= str(lines)
    newinput["inst_index"] = str(random.randint(9999,100000)) #radom 5 figure number
    newinput["domain"] = str(bik_df['Citation'][i])
    newinput["url"] = bik_df['URL'][i]
    newinput["summary"] = "" #Blank until we figure out the abstract stuff
    newinput['authors'] = str([bik_df['Authors'][i]])
    newinput['date'] =  '03-28-' + str(bik_df['Year'][i])  #Needs actual published date in form of 'M-D-2022' but try with today + publushed year
    newinput["split"] = "info"
    newinput["status"] = "succcess"
    
    master_file.append(newinput)


In [31]:
with open('gen_text.jsonl','w', encoding='utf-8') as f:
    for item in master_file:
        f.write(json.dumps(item) + "\n")